In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings 
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv(r"E:\Data Science by SRK\Machine_learning\Classification\Diabetes\cleaned_dataset_diabetes.csv")

In [3]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,89.0,66.0,23.0,1.460250,28.1,0.861443,1.288798,0
1,0,137.0,40.0,35.0,1.532648,43.1,1.071407,1.338267,1
2,3,78.0,50.0,32.0,1.452246,31.0,0.890303,1.311941,1
3,2,197.0,70.0,45.0,1.690052,30.5,0.857475,1.392162,1
4,1,189.0,60.0,23.0,1.753669,30.1,0.926098,1.404659,1
...,...,...,...,...,...,...,...,...,...
387,0,181.0,88.0,44.0,1.681244,43.3,0.882124,1.311941,1
388,1,128.0,88.0,39.0,1.479504,36.5,1.004630,1.351088,1
389,2,88.0,58.0,26.0,1.259921,28.4,0.978030,1.293804,0
390,10,101.0,76.0,48.0,1.541485,32.9,0.863144,1.412359,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               392 non-null    int64  
 1   Glucose                   392 non-null    float64
 2   BloodPressure             392 non-null    float64
 3   SkinThickness             392 non-null    float64
 4   Insulin                   392 non-null    float64
 5   BMI                       392 non-null    float64
 6   DiabetesPedigreeFunction  392 non-null    float64
 7   Age                       392 non-null    float64
 8   Outcome                   392 non-null    int64  
dtypes: float64(7), int64(2)
memory usage: 27.7 KB


In [5]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

# X and y

In [6]:
X = df.drop(columns = ['Outcome'])
y = df['Outcome']

# Train_test_split

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= True)

# ADABOOST
**Applying Hyperparameter Tuning for identifying best parameters for adaboost**

In [8]:
from sklearn.model_selection import GridSearchCV

# modelling

from sklearn.ensemble import AdaBoostClassifier
estimator_ab = AdaBoostClassifier()

# parameters grid
param_grid_ab = {"n_estimators" : list(range(1,100))}

# grid search
grid_ab = GridSearchCV(estimator_ab, param_grid_ab, cv = 5, scoring = 'accuracy')
grid_ab.fit(X_train, y_train)

# best parameters for adaboost model
grid_ab.best_params_

{'n_estimators': 5}

# ADA Boost model important features

In [9]:
feats_ab = pd.DataFrame(data = grid_ab.best_estimator_.feature_importances_,
                       index = X.columns, 
                       columns = ['Importance'])
imp_features_list_ab = feats_ab[feats_ab['Importance']>0].index.to_list()
imp_features_list_ab

['Glucose', 'BMI', 'DiabetesPedigreeFunction', 'Age']

# ADABoost Classifier

**with imortant features and best hyperparameters**

In [10]:
# important input variables
X_imp = X[imp_features_list_ab]

# train test split
X_train_ab, X_test_ab, y_train_ab, y_test_ab = train_test_split(X_imp, y, test_size = 0.2, random_state = True)

# modelling
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators= 5)
ada.fit(X_train_ab, y_train_ab)

# Evaluation on train data
ypred_train_ab = ada.predict(X_train_ab)

from sklearn.metrics import accuracy_score
print("Train_accuracy : ", accuracy_score(y_train_ab, ypred_train_ab))

# cross validation
from sklearn.model_selection import cross_val_score
print("cross_val_score : ", cross_val_score(ada, X_train_ab, y_train_ab, cv = 5).mean())

# Evaluation
ypred_test_ab = ada.predict(X_test_ab)
print("Test_accuracy : ", accuracy_score(y_test_ab, ypred_test_ab))


Train_accuracy :  0.8178913738019169
cross_val_score :  0.8051203277009729
Test_accuracy :  0.7468354430379747


In [11]:
%%timeit
ada.fit(X_train_ab, y_train_ab)


10.5 ms ± 773 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
